
# Using the Decision Transformer model with 🤗 Transformers

* [HF-Decision Transformer-Blog](https://huggingface.co/blog/decision-transformers)
* [HF-Train Decision Transformer-Blog](https://huggingface.co/blog/train-decision-transformers)

# Decision Transformer

The main idea of decision transformer is that instead of training a policy using RL methods, such as fitting a value function that will tell us what action to take to maximize the return (cumulative reward), we use a **sequence modeling algorithm (Transformer)** that, given the desired return, past states, and actions, will generate future actions to achieve this desired return. It’s an autoregressive model conditioned on the desired return, past states, and actions to generate future actions that achieve the desired return.

This is a complete shift in the Reinforcement Learning paradigm since we use **generative trajectory modeling** (modeling the joint distribution of the sequence of states, actions, and rewards) to replace conventional RL algorithms. It means that in Decision Transformers, we don’t maximize the return but rather generate a series of future actions that achieve the desired return.

**The process goes this way:**

* We feed **the last K timesteps** into the Decision Transformer with three inputs:
    * Return-to-go
    * State
    * Action
* **The tokens are embedded** either with a linear layer if the state is a vector or a CNN encoder if it’s frames.
* **The inputs are processed by a GPT-2 model**, which predicts future actions via autoregressive modeling.

<center>
    <img src="https://huggingface.co/blog/assets/58_decision-transformers/dt-architecture.gif"/>
</center>



## Installing dependencies
We first need to install the required dependencies in order to run a Mujoco environment inside a Colab notebook




In [1]:
!pip install gym
!pip install gym[mujoco]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 17.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.4 MB/s eta 0:00:00:00:01


In [2]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common \
    patchelf \
    xvfb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgl1-mesa-glx is already the newest version (21.2.6-0ubuntu0.1~20.04.2).
software-properties-common is already the newest version (0.99.9.12).
xvfb is already the newest version (2:1.20.13-1ubuntu1~20.04.17).
The following additional packages will be installed:
  libegl-dev libegl-mesa0 libegl1 libgbm1 libgl-dev libgles-dev libgles1
  libgles2 libglew2.1 libglu1-mesa libglu1-mesa-dev libglvnd-dev libglx-dev
  libopengl-dev libopengl0 libosmesa6 libpthread-stubs0-dev libwayland-server0
  libx11-dev libxau-dev libxcb1-dev libxdmcp-dev x11proto-core-dev
  x11proto-dev xorg-sgml-doctools xtrans-dev
Suggested packages:
  glew-utils libx11-doc libxcb-doc
The following NEW packages will be installed:
  libegl-dev libegl-mesa0 libegl1 libgbm1 libgl-dev libgl1-mesa-dev
  libgles-dev libgles1 libgles2 libglew-dev libglew2.1 libglu1-mesa
  libglu1-mesa-dev libglvnd-dev libglx-dev libopengl-dev libope

## Installing Pip packages
We also require the following pip packages:

In [3]:
!pip install git+https://github.com/huggingface/transformers

!pip install colabgymrender
!pip install xvfbwrapper
!pip install imageio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ojhpox25
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ojhpox25
  Resolved https://github.com/huggingface/transformers to commit 1082361a1978d30db5c3932d1ee08914d74d9697
  Installing build dependencies ... |^C
anceled
ERROR: Operation cancelled by user
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 7.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/26.9 MB 136.2 MB/s eta 0:00:01^C
   ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/26.9 MB 135.5 MB/s eta 0:00:01
ERROR: Operation cancelled by user
^C
ERROR: Operation cancelled by user
^C
ERROR: Operation cancelled by user


In [3]:
!pip3 install pyvirtualdisplay

In [ ]:
import os
os.kill(os.getpid(), 9)

In [2]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

# Part I : Using Pretrained Decision Transformer Model

## Importing the packages


In [3]:
import torch
# import mujoco_py
# import mujoco
import gym
import numpy as np

from colabgymrender.recorder import Recorder
from transformers import DecisionTransformerModel

## Defining a function that performs masked autoregressive predictive of actions.

The model's prediction is conditioned on sequences of states, actions, time-steps and returns. The action for the current time-step is included as zeros and masked in to not skew the model's attention distribution.

In [7]:
# Function that gets an action from the model using autoregressive prediction with a window of the 
# previous 20 timesteps.
def get_action(model, states, actions, rewards, returns_to_go, timesteps):
    # This implementation does not condition on past rewards

    states = states.reshape(1, -1, model.config.state_dim)
    actions = actions.reshape(1, -1, model.config.act_dim)
    returns_to_go = returns_to_go.reshape(1, -1, 1)
    timesteps = timesteps.reshape(1, -1)

    states = states[:, -model.config.max_length :]
    actions = actions[:, -model.config.max_length :]
    returns_to_go = returns_to_go[:, -model.config.max_length :]
    timesteps = timesteps[:, -model.config.max_length :]
    
    # pad all tokens to sequence length
    padding = model.config.max_length - states.shape[1]
    attention_mask = torch.cat([torch.zeros(padding), torch.ones(states.shape[1])])
    attention_mask = attention_mask.to(dtype=torch.long).reshape(1, -1)
    
    states = torch.cat([torch.zeros((1, padding, model.config.state_dim)), states], dim=1).float()
    actions = torch.cat([torch.zeros((1, padding, model.config.act_dim)), actions], dim=1).float()
    returns_to_go = torch.cat([torch.zeros((1, padding, 1)), returns_to_go], dim=1).float()
    timesteps = torch.cat([torch.zeros((1, padding), dtype=torch.long), timesteps], dim=1)

    state_preds, action_preds, return_preds = model(
        states=states,
        actions=actions,
        rewards=rewards,
        returns_to_go=returns_to_go,
        timesteps=timesteps,
        attention_mask=attention_mask,
        return_dict=False,
    )

    return action_preds[0, -1]

## Environment 

In [8]:
# build the environment
directory = './video/hopper'
env = gym.make("Hopper-v4", render_mode='rgb_array')
# env = Recorder(env, directory, fps=30)
env = gym.wrappers.RecordVideo(env, directory)
max_ep_len = 1000
device = "cpu"
scale = 1000.0  # normalization for rewards/returns
TARGET_RETURN = 3600.0 / scale  # evaluation is conditioned on a return of 3600, scaled accordingly

# mean and std computed from training dataset these are available in the model card for each model.
state_mean = np.array(
    [1.3490015,  -0.11208222, -0.5506444,  -0.13188992, -0.00378754,  2.6071432,
     0.02322114, -0.01626922, -0.06840388, -0.05183131,  0.04272673,]
)
state_std = np.array(
    [0.15980862, 0.0446214,  0.14307782, 0.17629202, 0.5912333,  0.5899924,
 1.5405099,  0.8152689,  2.0173461,  2.4107876,  5.8440027,]
)

state_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]

# Create the decision transformer model
model = DecisionTransformerModel.from_pretrained("edbeeching/decision-transformer-gym-hopper-expert")
model = model.to(device)
print(list(model.encoder.wpe.parameters()))

state_mean = torch.from_numpy(state_mean).to(device=device)
state_std = torch.from_numpy(state_std).to(device=device)

Moviepy - Building video /kaggle/working/video/hopper/rl-video-episode-0.mp4.
Moviepy - Writing video /kaggle/working/video/hopper/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /kaggle/working/video/hopper/rl-video-episode-0.mp4


[Parameter containing:
tensor([[-0., -0., 0.,  ..., 0., 0., 0.],
        [0., -0., -0.,  ..., 0., 0., -0.],
        [-0., 0., -0.,  ..., 0., 0., -0.],
        ...,
        [0., -0., -0.,  ..., 0., 0., 0.],
        [0., -0., 0.,  ..., 0., 0., 0.],
        [0., -0., 0.,  ..., 0., 0., -0.]], requires_grad=True)]


## Interaction through Pre-Trained Model 

In [9]:
# Interact with the environment and create a video
episode_return, episode_length = 0, 0
state, _ = env.reset()
target_return = torch.tensor(TARGET_RETURN, device=device, dtype=torch.float32).reshape(1, 1)
states = torch.from_numpy(state).reshape(1, state_dim).to(device=device, dtype=torch.float32)
actions = torch.zeros((0, act_dim), device=device, dtype=torch.float32)
rewards = torch.zeros(0, device=device, dtype=torch.float32)

timesteps = torch.tensor(0, device=device, dtype=torch.long).reshape(1, 1)

for t in range(max_ep_len):
    actions = torch.cat([actions, torch.zeros((1, act_dim), device=device)], dim=0)
    rewards = torch.cat([rewards, torch.zeros(1, device=device)])

    action = get_action(
        model,
        (states - state_mean) / state_std,
        actions,
        rewards,
        target_return,
        timesteps,
    )
    actions[-1] = action
    action = action.detach().cpu().numpy()

    state, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated

    cur_state = torch.from_numpy(state).to(device=device).reshape(1, state_dim)
    states = torch.cat([states, cur_state], dim=0)
    rewards[-1] = reward

    pred_return = target_return[0, -1] - (reward / scale)
    target_return = torch.cat([target_return, pred_return.reshape(1, 1)], dim=1)
    timesteps = torch.cat([timesteps, torch.ones((1, 1), device=device, dtype=torch.long) * (t + 1)], dim=1)

    episode_return += reward
    episode_length += 1

    if done:
        break

Moviepy - Building video /kaggle/working/video/hopper/rl-video-episode-0.mp4.
Moviepy - Writing video /kaggle/working/video/hopper/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /kaggle/working/video/hopper/rl-video-episode-0.mp4


In [11]:
from IPython.display import HTML

from pathlib import Path
import datetime
import json
import imageio
import IPython
import base64
import os

def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="480" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())
    return IPython.display.HTML(tag)

os.system('ffmpeg -i /kaggle/working/video/hopper/rl-video-episode-0.mp4 replay.mp4 -y')
embed_mp4('/kaggle/working/replay.mp4')

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

# Part II: Train Decision Transformer

In this tutorial, **you’ll learn to train your first Offline Decision Transformer model from scratch to make a half-cheetah run.** 🏃

❓ If you have questions, please post them on #study-group discord channel 👉 https://discord.gg/aYka4Yhff9

🎮 Environments:
- [Half Cheetah](https://www.gymlibrary.dev/environments/mujoco/half_cheetah/)

⬇️ Here's what you'll achieve at the end of this tutorial. ⬇️

In [6]:
%%html
<video controls autoplay><source src="https://huggingface.co/edbeeching/decision-transformer-gym-halfcheetah-expert/resolve/main/replay.mp4" type="video/mp4"></video>

### Prerequisites 🏗️
Before diving into the notebook, you need to:

🔲 📚 [Read the tutorial](https://huggingface.co/blog/train-decision-transformers)

## Step 0: Import 🔽

In [31]:
!pip install transformers
!pip install datasets
!pip install imageio
!pip install imageio-ffmpeg
!pip install huggingface_hub
!pip install xvfbwrapper
!pip install pyglet==1.5.1

!pip install moviepy==1.0.3

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [32]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [42]:
import os
import random
from dataclasses import dataclass

import gym
import numpy as np
import torch
from datasets import load_dataset
from transformers import DecisionTransformerConfig, DecisionTransformerModel, Trainer, TrainingArguments, AutoModel, AutoConfig

# Step 1: Loading the dataset from the 🤗 Hub

We host a number of Offline RL Datasets on the hub. Today we will be training with the halfcheetah “expert” dataset, hosted here on hub.

First we need to import the load_dataset function from the 🤗 datasets package and download the dataset to our machine.

In [34]:
os.environ["WANDB_DISABLED"] = "true" # we disable weights and biases logging for this tutorial
dataset = load_dataset("edbeeching/decision_transformer_gym_replay", "halfcheetah-expert-v2")

# Step 2: Defining a Custom DataCollator for transformers Trainer

In [35]:
@dataclass
class DecisionTransformerGymDataCollator:
    return_tensors: str = "pt"
    max_len: int = 20            # subsets of the episode we use for training
    state_dim: int = 17          # size of state space
    act_dim: int = 6             # size of action space
    max_ep_len: int = 1000       # max episode length in the dataset
    scale: float = 1000.0        # normalization of rewards/returns
    state_mean: np.array = None  # to store state means
    state_std: np.array = None   # to store state stds
    p_sample: np.array = None    # a distribution to take account trajectory lengths
    n_traj: int = 0              # to store the number of trajectories in the dataset

    def __init__(self, dataset) -> None:
        self.act_dim = len(dataset[0]["actions"][0])
        self.state_dim = len(dataset[0]["observations"][0])
        self.dataset = dataset
        
        # calculate dataset stats for normalization of states
        states = []
        traj_lens = []
        for obs in dataset["observations"]:
            states.extend(obs)
            traj_lens.append(len(obs))
        self.n_traj = len(traj_lens)
        states = np.vstack(states)
        self.state_mean, self.state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
        
        traj_lens = np.array(traj_lens)
        self.p_sample = traj_lens / sum(traj_lens)

    def _discount_cumsum(self, x, gamma):
        discount_cumsum = np.zeros_like(x)
        discount_cumsum[-1] = x[-1]
        for t in reversed(range(x.shape[0] - 1)):
            discount_cumsum[t] = x[t] + gamma * discount_cumsum[t + 1]
        return discount_cumsum

    def __call__(self, features):
        batch_size = len(features)
        
        # this is a bit of a hack to be able to sample of a non-uniform distribution
        batch_inds = np.random.choice(
            np.arange(self.n_traj),
            size=batch_size,
            replace=True,
            p=self.p_sample    # reweights so we sample according to timesteps
        )
        
        # a batch of dataset features
        s, a, r, d, rtg, timesteps, mask = [], [], [], [], [], [], []
        
        for ind in batch_inds:
            # for feature in features:
            feature = self.dataset[int(ind)]
            si = random.randint(0, len(feature["rewards"]) - 1)

            # get sequences from dataset
            s.append(np.array(feature["observations"][si : si + self.max_len]).reshape(1, -1, self.state_dim))
            a.append(np.array(feature["actions"][si : si + self.max_len]).reshape(1, -1, self.act_dim))
            r.append(np.array(feature["rewards"][si : si + self.max_len]).reshape(1, -1, 1))
            d.append(np.array(feature["dones"][si : si + self.max_len]).reshape(1, -1))
            timesteps.append(np.arange(si, si + s[-1].shape[1]).reshape(1, -1))
            timesteps[-1][timesteps[-1] >= self.max_ep_len] = self.max_ep_len - 1  # padding cutoff
            rtg.append(
                self._discount_cumsum(np.array(feature["rewards"][si:]), gamma=1.0)[
                    : s[-1].shape[1]   # TODO check the +1 removed here
                ].reshape(1, -1, 1)
            )
            if rtg[-1].shape[1] < s[-1].shape[1]:
                print("if true")
                rtg[-1] = np.concatenate([rtg[-1], np.zeros((1, 1, 1))], axis=1)

            # padding and state + reward normalization
            tlen = s[-1].shape[1]
            s[-1] = np.concatenate([np.zeros((1, self.max_len - tlen, self.state_dim)), s[-1]], axis=1)
            s[-1] = (s[-1] - self.state_mean) / self.state_std
            a[-1] = np.concatenate(
                [np.ones((1, self.max_len - tlen, self.act_dim)) * -10.0, a[-1]],
                axis=1,
            )
            r[-1] = np.concatenate([np.zeros((1, self.max_len - tlen, 1)), r[-1]], axis=1)
            d[-1] = np.concatenate([np.ones((1, self.max_len - tlen)) * 2, d[-1]], axis=1)
            rtg[-1] = np.concatenate([np.zeros((1, self.max_len - tlen, 1)), rtg[-1]], axis=1) / self.scale
            timesteps[-1] = np.concatenate([np.zeros((1, self.max_len - tlen)), timesteps[-1]], axis=1)
            mask.append(np.concatenate([np.zeros((1, self.max_len - tlen)), np.ones((1, tlen))], axis=1))

        # convert lists to tensors
        s = torch.from_numpy(np.concatenate(s, axis=0)).float()
        a = torch.from_numpy(np.concatenate(a, axis=0)).float()
        r = torch.from_numpy(np.concatenate(r, axis=0)).float()
        d = torch.from_numpy(np.concatenate(d, axis=0))
        rtg = torch.from_numpy(np.concatenate(rtg, axis=0)).float()
        timesteps = torch.from_numpy(np.concatenate(timesteps, axis=0)).long()
        mask = torch.from_numpy(np.concatenate(mask, axis=0)).float()

        return {
            "states": s,
            "actions": a,
            "rewards": r,
            "returns_to_go": rtg,
            "timesteps": timesteps,
            "attention_mask": mask,
        }

# Step 3: Extending the Decision Transformer Model to include a loss function

In order to train the model with the 🤗 trainer class, we first need to ensure the dictionary it returns contains a loss, in this case L-2 norm of the models action predictions and the targets.

In [36]:
class TrainableDT(DecisionTransformerModel):
    def __init__(self, config):
        super().__init__(config)

    def forward(self, **kwargs):
        output = super().forward(**kwargs)
        # add the DT loss
        action_preds = output[1]
        action_targets = kwargs["actions"]
        attention_mask = kwargs["attention_mask"]
        act_dim = action_preds.shape[2]
        action_preds = action_preds.reshape(-1, act_dim)[attention_mask.reshape(-1) > 0]
        action_targets = action_targets.reshape(-1, act_dim)[attention_mask.reshape(-1) > 0]
        
        loss = torch.mean((action_preds - action_targets) ** 2)

        return {"loss": loss}

    def original_forward(self, **kwargs):
        return super().forward(**kwargs)

In [55]:
collator = DecisionTransformerGymDataCollator(dataset["train"])

# config = DecisionTransformerConfig(state_dim=collator.state_dim, act_dim=collator.act_dim)
config = AutoConfig.from_pretrained('/kaggle/working/output')
model = TrainableDT(config)

# Step 4: Define training hyperparameters and train the model
Here, we define the training hyperparameters and our Trainer class that we'll use to train our Decision Transformer model.


In [56]:
training_args = TrainingArguments(
    output_dir="output/",
    remove_unused_columns=False,
    num_train_epochs=240,
    per_device_train_batch_size=64,
    learning_rate=1e-4,
    weight_decay=1e-4,
    warmup_ratio=0.1,
    optim="adamw_torch",
    max_grad_norm=0.25,
#     load_best_model_at_end=True
)

# model = AutoModel.from_pretrained("/kaggle/working/output")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    data_collator=collator,
)

# trainer.train(resume_from_checkpoint=True)   # using this enables trainer to resume training from last checkpoint

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


# Step 5: Visualize the performance of the agent

## Get Action Function 

In [51]:
# Function that gets an action from the model using autoregressive prediction with 
# a window of the previous 20 timesteps.
def get_action(model, states, actions, rewards, returns_to_go, timesteps):
    # This implementation does not condition on past rewards

    states = states.reshape(1, -1, model.config.state_dim)
    actions = actions.reshape(1, -1, model.config.act_dim)
    returns_to_go = returns_to_go.reshape(1, -1, 1)
    timesteps = timesteps.reshape(1, -1)

    states = states[:, -model.config.max_length :]
    actions = actions[:, -model.config.max_length :]
    returns_to_go = returns_to_go[:, -model.config.max_length :]
    timesteps = timesteps[:, -model.config.max_length :]
    padding = model.config.max_length - states.shape[1]
    
    # pad all tokens to sequence length
    attention_mask = torch.cat([torch.zeros(padding), torch.ones(states.shape[1])])
    attention_mask = attention_mask.to(dtype=torch.long).reshape(1, -1)
    
    states = torch.cat([torch.zeros((1, padding, model.config.state_dim)), states], dim=1).float()
    actions = torch.cat([torch.zeros((1, padding, model.config.act_dim)), actions], dim=1).float()
    returns_to_go = torch.cat([torch.zeros((1, padding, 1)), returns_to_go], dim=1).float()
    timesteps = torch.cat([torch.zeros((1, padding), dtype=torch.long), timesteps], dim=1)

    state_preds, action_preds, return_preds = model.original_forward(
        states=states,
        actions=actions,
        rewards=rewards,
        returns_to_go=returns_to_go,
        timesteps=timesteps,
        attention_mask=attention_mask,
        return_dict=False,
    )

    return action_preds[0, -1]

## Build Environment 

In [64]:
directory = './video/halfcheetah'
model = model.to("cpu")
env = gym.make("HalfCheetah-v4", render_mode="rgb_array")
env = gym.wrappers.RecordVideo(env, directory)
max_ep_len = 1000
device = "cpu"
scale = 1000.0  # normalization for rewards/returns
TARGET_RETURN = 12000 / scale  # evaluation is conditioned on a return of 12000, scaled accordingly

state_mean = collator.state_mean.astype(np.float32)
state_std = collator.state_std.astype(np.float32)
print(state_mean)
print(state_std)

state_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]

state_mean = torch.from_numpy(state_mean).to(device=device)
state_std = torch.from_numpy(state_std).to(device=device)

## Interact With Env & Create Video 

In [65]:
episode_return, episode_length = 0, 0
state, _ = env.reset()
target_return = torch.tensor(TARGET_RETURN, device=device, dtype=torch.float32).reshape(1, 1)
states = torch.from_numpy(state).reshape(1, state_dim).to(device=device, dtype=torch.float32)
actions = torch.zeros((0, act_dim), device=device, dtype=torch.float32)
rewards = torch.zeros(0, device=device, dtype=torch.float32)

timesteps = torch.tensor(0, device=device, dtype=torch.long).reshape(1, 1)
for t in range(max_ep_len):
    actions = torch.cat([actions, torch.zeros((1, act_dim), device=device)], dim=0)
    rewards = torch.cat([rewards, torch.zeros(1, device=device)])

    action = get_action(
        model,
        (states - state_mean) / state_std,
        actions,
        rewards,
        target_return,
        timesteps,
    )
    actions[-1] = action
    action = action.detach().cpu().numpy()

    state, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated

    cur_state = torch.from_numpy(state).to(device=device).reshape(1, state_dim)
    states = torch.cat([states, cur_state], dim=0)
    rewards[-1] = reward

    pred_return = target_return[0, -1] - (reward / scale)
    target_return = torch.cat([target_return, pred_return.reshape(1, 1)], dim=1)
    timesteps = torch.cat([timesteps, torch.ones((1, 1), device=device, dtype=torch.long) * (t + 1)], dim=1)

    episode_return += reward
    episode_length += 1

    if done:
        break

Moviepy - Building video /kaggle/working/video/halfcheetah/rl-video-episode-0.mp4.
Moviepy - Writing video /kaggle/working/video/halfcheetah/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /kaggle/working/video/halfcheetah/rl-video-episode-0.mp4


In [66]:
from IPython.display import HTML

from pathlib import Path
import datetime
import json
import imageio
import IPython
import base64
import os

def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="480" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())
    return IPython.display.HTML(tag)

os.system('ffmpeg -i /kaggle/working/video/halfcheetah/rl-video-episode-0.mp4 /kaggle/working/output/replay.mp4 -y')
embed_mp4('/kaggle/working/output/replay.mp4')

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

# Step 6: Publish our trained model on the Hub 🔥
Now that we saw we got good results after the training, we can publish our trained model on the hub 🤗 with one line of code.

Under the hood, the Hub uses git-based repositories (don't worry if you don't know what git is), which means you can update the model with new versions as you experiment and improve your agent.

## HF Login 

In [16]:
from huggingface_hub import notebook_login
notebook_login()
!git config --global credential.helper store

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


## Final Push 

In [62]:
kwargs = {
    "dataset_tags": "edbeeching/decision_transformer_gym_replay",
    "dataset": "halfcheetah-expert-v2",  # a 'pretty' name for the training dataset
    "model_name": "decision-transformer-halfcheetah-v4",  # a 'pretty' name for our model
}

args = training_args.set_push_to_hub("hishamcse/decision-transformer-halfcheetah-v4")

trainer.push_to_hub(**kwargs)

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

replay.mp4:   0%|          | 0.00/1.41M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.03M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hishamcse/decision-transformer-halfcheetah-v4/commit/e8053ffcf9ae6ce284840a22aedb8e298cfbde1a', commit_message='End of training', commit_description='', oid='e8053ffcf9ae6ce284840a22aedb8e298cfbde1a', pr_url=None, pr_revision=None, pr_num=None)